In [1]:
# Importar Bibliotecas

import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta

In [2]:
# Variáveis do Experimento

NUM_USERS = 100000
NUM_DAYS = 30
START_DATE = datetime(2025,1,1)

In [3]:
# Gerar um pequeno fator de "melhora" ou "piora" aleatória para o grupo de teste
# O fator será um número aleatório entre -0.01 (piora de 1%) e 0.05 (melhora de 5%)

random_lift_add_to_cart = random.uniform(-0.01, 0.05)
random_lift_purchase = random.uniform(-0.01, 0.05)

In [4]:
# Definir as probabilidades base

BASE_PROBABILITIES = {
    'view_page': 1.0,
    'add_to_cart': 0.10,  #10% dos usuários que visualizam, adicionam ao carrinho
    'purchase': 0.02  #2% dos usuários que adicionam, compram
}

In [5]:
# Probabilidades de conversão para cada grupo
# O grupo de teste recebe um "lift" aleatório

PROBABILITIES = {
    'control': BASE_PROBABILITIES,
    'test': {
        'view_page': 1.0,
        'add_to_cart': BASE_PROBABILITIES['add_to_cart'] + random_lift_add_to_cart,
        'purchase': BASE_PROBABILITIES['purchase'] + random_lift_purchase
    }
}

In [6]:
# Probabilidades de atributos de usuários e produtos

DEVICE_PROBS = [0.7, 0.25, 0.05]
DEVICE_TYPES = ['Desktop', 'Mobile', 'Tablet']
TRAFFIC_SOURCE_PROBS = [0.4, 0.3, 0.15, 0.15]
TRAFFIC_SOURCES = ['Organic', 'Paid', 'Email', 'Social']
LOCATIONS = ['São Paulo', 'Rio de Janeiro', 'Belo Horizonte', 'Porto Alegre']
PRODUCT_CATEGORIES = ['Eletronics', 'Fashion', 'Home', 'Books', 'Sports']

In [7]:
# Número de produtos no nosso catálogo

NUM_PRODUCTS = 500

In [8]:
# Vamos ver quais foram as probabilidades geradas para o grupo de teste
print("Probabilidades do Grupo de Teste (geradas aleatoriamente):")
print(f"  - Add to Cart: {PROBABILITIES['test']['add_to_cart']:.4f}")
print(f"  - Purchase: {PROBABILITIES['test']['purchase']:.4f}")

Probabilidades do Grupo de Teste (geradas aleatoriamente):
  - Add to Cart: 0.1359
  - Purchase: 0.0216


In [9]:
# Geração de IDs únicos para eventos
event_id_counter = 0
def generate_event_id():
    global event_id_counter
    event_id_counter += 1
    return event_id_counter

def simulate_user_journey(user_id, group, start_time, session_id):
    """
    Simula a jornada de um único usuário através do funil de vendas.
    """
    events = []
    
    # Probabilidades de conversão para o grupo do usuário
    group_probs = PROBABILITIES[group]
    
    # Geração de atributos aleatórios para o usuário
    device = np.random.choice(DEVICE_TYPES, p=DEVICE_PROBS)
    traffic_source = np.random.choice(TRAFFIC_SOURCES, p=TRAFFIC_SOURCE_PROBS)
    location = random.choice(LOCATIONS)
    
    # Passo 1: O usuário visualiza a página do produto
    if random.random() < group_probs['view_page']:

        # Simula o produto visualizado
        product_id = random.randint(1, NUM_PRODUCTS)
        product_category = random.choice(PRODUCT_CATEGORIES)
        product_price = round(random.uniform(20.0, 300.0), 2)
        
        event = {
            'event_id': generate_event_id(),
            'user_id': user_id,
            'group': group,
            'timestamp': start_time,
            'event_type': 'view_page',
            'session_id': session_id,
            'product_id': product_id,
            'product_category': product_category,
            'product_price': product_price,
            'quantity': None,
            'order_id': None,
            'total_value': None,
            'device_type': device,
            'traffic_source': traffic_source,
            'location': location
        }
        events.append(event)
        
        # Passo 2: O usuário adiciona ao carrinho (probabilidade dependente do grupo)
        if random.random() < group_probs['add_to_cart']:
            add_to_cart_time = start_time + timedelta(seconds=random.randint(30, 600))
            
            event = {
                'event_id': generate_event_id(),
                'user_id': user_id,
                'group': group,
                'timestamp': add_to_cart_time,
                'event_type': 'add_to_cart',
                'session_id': session_id,
                'product_id': product_id,
                'product_category': product_category,
                'product_price': product_price,
                'quantity': 1,
                'order_id': None,
                'total_value': None,
                'device_type': device,
                'traffic_source': traffic_source,
                'location': location
            }
            events.append(event)
            
            # Passo 3: O usuário finaliza a compra (probabilidade dependente do grupo)
            if random.random() < group_probs['purchase']:
                purchase_time = add_to_cart_time + timedelta(seconds=random.randint(60, 1200))
                order_id = random.randint(100000, 999999)
                total_value = round(product_price * 1.1, 2)
                
                event = {
                    'event_id': generate_event_id(),
                    'user_id': user_id,
                    'group': group,
                    'timestamp': purchase_time,
                    'event_type': 'purchase',
                    'session_id': session_id,
                    'product_id': product_id,
                    'product_category': product_category,
                    'product_price': product_price,
                    'quantity': 1,
                    'order_id': order_id,
                    'total_value': total_value,
                    'device_type': device,
                    'traffic_source': traffic_source,
                    'location': location
                }
                events.append(event)
                
    return events

In [10]:
# Lista para armazenar todos os eventos gerados
all_events = []

# --- Gerando dados para uma amostra de 5 usuários ---
for user_id in range(1, 6):  # Rodando a simulação para os 5 primeiros usuários
    # Atribuição aleatória de grupo para o teste A/B
    group = np.random.choice(['control', 'test'], p=[0.5, 0.5])
    
    # Geração de uma data e hora de início de sessão aleatória
    session_start_time = START_DATE + timedelta(days=random.randint(0, NUM_DAYS-1),
                                                 hours=random.randint(0, 23),
                                                 minutes=random.randint(0, 59))
    session_id = random.randint(1000, 9999) # ID de sessão aleatório

    # Simula a jornada do usuário e adiciona os eventos à lista
    user_events = simulate_user_journey(user_id, group, session_start_time, session_id)
    all_events.extend(user_events)

# Converte a lista de dicionários para um DataFrame do Pandas
df_sample = pd.DataFrame(all_events)

# Exibe o resultado para validação
print("Amostra do DataFrame (primeiros 5 usuários):")
display(df_sample)

Amostra do DataFrame (primeiros 5 usuários):


,event_id,user_id,group,timestamp,event_type,session_id,product_id,product_category,product_price,quantity,order_id,total_value,device_type,traffic_source,location
0,1,1,control,2025-01-05 10:38:00,view_page,7285,408,Eletronics,97.66,NaN,None,None,Mobile,Organic,Belo Horizonte
1,2,2,test,2025-01-29 02:12:00,view_page,6315,180,Sports,161.56,NaN,None,None,Desktop,Paid,Belo Horizonte
2,3,3,test,2025-01-16 08:18:00,view_page,8091,152,Home,26.86,NaN,None,None,Desktop,Organic,São Paulo
3,4,3,test,2025-01-16 08:19:11,add_to_cart,8091,152,Home,26.86,1.0,None,None,Desktop,Organic,São Paulo
4,5,4,test,2025-01-19 09:32:00,view_page,9843,275,Sports,244.38,NaN,None,None,Tablet,Email,Rio de Janeiro
5,6,4,test,2025-01-19 09:32:56,add_to_cart,9843,275,Sports,244.38,1.0,None,None,Tablet,Email,Rio de Janeiro
6,7,5,test,2025-01-14 18:48:00,view_page,7183,414,Sports,25.93,NaN,None,None,Desktop,Social,Rio de Janeiro


In [11]:
# Lista para armazenar todos os eventos gerados
all_events = []

# Gerando os dados para todos os  100.000 usuários
for user_id in range(1, NUM_USERS + 1):
    group = np.random.choice(['control', 'test'], p=[0.5, 0.5])
    session_start_time = START_DATE + timedelta(days=random.randint(0, NUM_DAYS - 1),
                                                hours=random.randint(0, 23),
                                                minutes=random.randint(0, 59))
    
    session_id = random.randint(100000, 999999)

    user_events = simulate_user_journey(user_id, group, session_start_time, session_id)
    all_events.extend(user_events)

# Cria o DataFrame final
df = pd.DataFrame(all_events)

# Exibe as primeiras linhas e o tamanho total do DataFrame
print("DataFrame Final:")
display(df.head())
print(f"\nNúmero total de eventos gerados: {len(df)}")

DataFrame Final:


,event_id,user_id,group,timestamp,event_type,session_id,product_id,product_category,product_price,quantity,order_id,total_value,device_type,traffic_source,location
0,8,1,control,2025-01-12 07:31:00,view_page,960679,331,Sports,177.03,NaN,NaN,NaN,Desktop,Social,São Paulo
1,9,2,test,2025-01-30 23:46:00,view_page,433115,412,Fashion,183.68,NaN,NaN,NaN,Desktop,Organic,São Paulo
2,10,3,test,2025-01-18 22:41:00,view_page,621107,340,Fashion,213.50,NaN,NaN,NaN,Mobile,Organic,São Paulo
3,11,4,control,2025-01-22 13:19:00,view_page,153424,494,Home,279.20,NaN,NaN,NaN,Mobile,Email,Rio de Janeiro
4,12,5,control,2025-01-07 13:43:00,view_page,242805,262,Eletronics,250.01,NaN,NaN,NaN,Desktop,Paid,Belo Horizonte



Número total de eventos gerados: 112191


In [12]:
# Salva o DataFrame em um arquivo CSV

df.to_csv('ecommerce_events_data.csv', index=False)

print("\nBase de dados salva com sucesso como 'ecommerce_events_data.csv'")


Base de dados salva com sucesso como 'ecommerce_events_data.csv'
